In [16]:
from sklearn.cluster import KMeans
import pandas as pd
import seaborn as sns
import numpy as np

## Data Loading

In [3]:
biomass_density = pd.read_csv('preprocessed/forecasted.csv')

In [6]:
distance_matrix = pd.read_csv('dataset/Distance_Matrix.csv')

In [102]:
distance_matrix=distance_matrix.drop('Unnamed: 0',axis=1)

In [103]:
distance_matrix.shape

(2418, 2418)

## Prprocessing

In [8]:
#new_distance = lambda den,dist: den/dist

In [9]:
distance_matrix.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417
0,0,0.0000,11.3769,20.4557,38.1227,45.3810,54.9915,78.6108,118.6750,102.6639,...,683.8771,687.6310,697.3246,669.3962,667.6788,665.5775,662.0291,665.9655,673.2073,681.4235
1,1,11.3769,0.0000,9.0788,28.9141,36.1724,45.7829,69.4022,78.2329,93.4553,...,681.6295,685.3833,695.0769,667.1485,665.4311,663.3298,659.7815,663.7178,670.9596,679.1758
2,2,20.4557,9.0788,0.0000,22.3791,29.6374,39.2478,62.8671,71.6979,86.9203,...,682.2323,685.9861,695.6796,667.7513,666.0339,663.9326,660.3843,664.3206,671.5623,679.7786
3,3,38.1227,28.9141,22.3791,0.0000,11.8343,23.5413,41.8396,50.6703,65.8927,...,681.4226,685.1765,694.8701,666.9417,665.2243,663.1230,659.5746,663.5110,670.7528,678.9690
4,4,45.3810,36.1724,29.6374,11.8343,0.0000,11.7070,24.3986,33.2293,53.9901,...,663.9816,667.7355,677.4291,649.5007,647.7833,645.6820,642.1336,646.0700,653.3118,661.5280


## Modelling

### Get initial point with most density of biomass

In [24]:
def get_most(column_name):
    density=biomass_density[column_name]
    return np.argmax(density)

### Get pobability based on biomass density

In [ ]:
k=3

In [27]:
def set_prob(density):
    sum_density=sum(density)
    prob=[x/sum_density for x in density]
    return prob

In [35]:
weights=set_prob(biomass_density['2018'])

In [105]:
def get_centers(k,column_name,weights):
    centers=[]
    while len(centers)!=k :
        centers = np.random.choice(biomass_density[column_name].shape[0],size=k,p=weights)
    return centers

In [116]:
biomass_density['2018'].shape[0]

2418

In [107]:
centers = get_centers(k,'2018',weights)

In [142]:
def re_cluster(k,X,centers):
    belonging_to_cluster=np.array([])
    for i in range(X.shape[0]):
        distance=[distance_matrix.iloc[i,x] for x in centers]
        belonging_to_cluster=np.append(belonging_to_cluster,np.argmin(distance))
    
    for i in range(k):
        pseudo_distance = np.mean([distance_matrix.iloc[i,x] for x in range(X.shape[0])])
        print(pseudo_distance)
        
    return belonging_to_cluster

In [143]:
c=re_cluster(k,biomass_density,centers)
np.unique(c,return_counts=True)

367.87483995037223
365.5726327956989
369.27432857733663


(array([0., 1., 2.]), array([ 329,  756, 1333]))